In [1]:
import sys, os, time
import shutil
%pdb
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision

from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import imageio
import seaborn as sns
# plt.switch_backend('Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from bisect import bisect_left

# from IPython.display import HTML, Image

#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data
from analyze import *

from tqdm import tqdm_notebook as tqdm

# plt.ioff()

np.set_printoptions(linewidth=120, suppress=True)

Automatic pdb calling has been turned ON


In [2]:
media_dir = '/is/ei/fleeb/workspace/media/hybrid/results'

In [3]:
M = Hybrid_Controller(media_dir).sort_by('date')

Found 27 runs


In [4]:
M.clear_filters()
pass

In [5]:
M.prep_info(name='model.pth.tar')

M.load_into('eval.pth.tar')

Selecting checkpoint: last


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:42: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Loaded configs


In [6]:
def _get_stats(run):
    if 'eval' in run:
        run.stats = {
            'FID-prior': run.eval['FID-prior'],
            
            
#             'DCI-inf': run.eval['DCI']['informativeness_test'],
#             'DCI-cmp': run.eval['DCI']['completeness'],
        }
        
        if 'FID-hyb' in run.eval and run.eval['FID-hyb'] is not None:
            run.stats.update({
                'FID-hyb': run.eval['FID-hyb'],
                'FID-rec': run.eval['FID-rec'],
            })
        
        if 'IRS' in run.eval and run.eval['IRS'] is not None:
            run.stats.update({
                
                'IRS': run.eval['IRS']['IRS'],
                'DCI-dis': run.eval['DCI']['disentanglement'],
            })
M.map(_get_stats, safe=True)

M.filter(lambda r: 'stats' in r and len(r.stats) and 'FID-hyb' in r.stats)
pass

M.store_selection('done')


In [7]:
M.show('all')

  0) 3dshapes-ae_0002-6026518-03_200129-001602_ckpt40
	Checkpoint: 40 (approx 120000 steps)
	 FID-prior : 51.53
	   FID-hyb : 34.46
	   FID-rec : 23.75
	       IRS : 0.56
	   DCI-dis : 0.28
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 0 (100)

  1) 3dshapes-dvae_0004-6026551-05_200129-003640_ckpt24
	Checkpoint: 24 (approx 72000 steps)
	 FID-prior : 89.82
	   FID-hyb : 83.77
	   FID-rec : 68.20
	       IRS : 0.56
	   DCI-dis : 0.34
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.prob - 0.1 (0.5)

  2) 3dshapes-dvae_0004-6026551-06_200129-003638_ckpt24
	Checkpoint: 24 (approx 72000 steps)
	 FID-prior : 75.81
	   FID-hyb : 69.75
	   FID-rec : 56.87
	       IRS : 0.65
	   DCI-dis : 0.39
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.prob - 0.2 (0.5)

  3) 3dshapes-dvae_0004-6026551-07_200129-003721_ckpt23
	Checkpoint: 23 (approx 69000 steps)
	 FID-prior : 70.69
	   FID-hyb : 68.80
	   FID-rec : 57.3

In [8]:
M.switch_selection('done')
len(M)

25

In [9]:
# M.invert()
# len(M)
# M.show()
# ' '.join(['_'.join(run.name.split('_')[:-1]) for run in M])

In [10]:
plt.close('all')
figaxs = {}

In [11]:
M.map(lambda r: r.meta.job[0], reduce=Counter)

Counter({'2': 9, '4': 8, '0': 7, '3': 1})

In [15]:
M.switch_selection('done')

# M.filter(lambda r: 'arch' in r.meta and r.meta.arch != 'double')

# M.sort_by(lambda runs: util.sort_by(runs, [run.stats['IRS'] for run in runs]))
M.sort_by(lambda runs: util.sort_by(runs, [run.stats['FID-rec'] for run in runs]))
# M.filter_sel(slice(-10,None,None))


M.filter_models('fdwae', 'dvae', 'dwae')
# M.filter_jobs(94, 122)
# M.show('checkpoint', 'unique', )
# M.filter(lambda r: r.stats['DCI-dis'] > 0.75)

pass

In [16]:
fig_stats = {
    'dis':['DCI-dis', 'IRS'], 
    'fid2': ['FID-rec', 'FID-hyb'],
    'fid': ['FID-prior', ],
#              'fid':['FID-rec', ],
            }
for i, (name,stats,) in enumerate(fig_stats.items()):
    if name not in figaxs:
        figaxs[name] = plt.subplots(figsize=(12,5))
    figaxs[name][1].cla()
    figaxs[name] = M.plot_stats(*stats, figax=figaxs.get(name,None))#, color='C{}'.format(i))
    plt.tight_layout()
# M.show('checkpoint', 'unique', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])
M.show('all', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])

  0) 3dshapes-fdwae_0000-6026515-03_200129-001058_ckpt24
	Checkpoint: 24 (approx 72000 steps)
	 FID-prior : 61.54
	   FID-hyb : 58.22
	   FID-rec : 48.19
	       IRS : 0.64
	   DCI-dis : 0.42
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 1000 (100)
	model.reg_prior - 0.1 (0.5)
	model.prob - 0.1 (0.5)
	Generator: double-dec

  1) 3dshapes-fdwae_0000-6026515-05_200129-000937_ckpt23
	Checkpoint: 23 (approx 69000 steps)
	 FID-prior : 64.98
	   FID-hyb : 58.60
	   FID-rec : 51.23
	       IRS : 0.68
	   DCI-dis : 0.50
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 1000 (100)
	model.gan_wt - 5e-05 (0.0001)
	model.reg_prior - 0.1 (0.5)
	Generator: double-dec

  2) 3dshapes-fdwae_0000-6026515-07_200129-001057_ckpt22
	Checkpoint: 22 (approx 66000 steps)
	 FID-prior : 70.52
	   FID-hyb : 64.25
	   FID-rec : 53.47
	       IRS : 0.67
	   DCI-dis : 0.48
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)


In [26]:
run = M[0]

In [27]:
run.config.model.generator._type

'double-dec'

In [28]:
run.stats

{'FID-prior': 51.52777621528335,
 'FID-hyb': 34.45774436178269,
 'FID-rec': 23.747642992132057,
 'IRS': 0.56331756810021,
 'DCI-dis': 0.2810307932603152}

In [30]:
run.meta.arch

'double'